In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

API_KEY = config['OANDA']['API_KEY'] 
ACCOUNT_ID = config['OANDA']['ACCOUNT_ID']
HOSTNAME   = "api-fxpractice.oanda.com"
STREAMHOST = "stream-fxpractice.oanda.com"
key = f'Bearer {API_KEY}'

# contexts
ctx = v20.Context(hostname=HOSTNAME, token=key)
ctx.set_header(key='Authorization', value=key)